In [2]:
from keras.models import Sequential, Model
from keras.layers import Flatten,Dense,Dropout
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D ,Input
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# VGG
# daha küçük filtreler kullanarak daha derin bir mimari elde etmek
# elde ettikleri mimariyi titanics işlemci üzerinden eğitiyorlar
# derinlikten kastı 16-19 katman arasında değişen modeller
# lokalizasyonda 1.  : nesnenin görüntünün içerisinde yerinin tespit edilmesi
# sınıflandırmada 2. olmuşlar  : nesne görüntünün içerisinde var mı yok mu.
# en meşhurları vgg-16 ve  vgg-19
# 133-135 milyon params
# yine 4096-4096-100lik çıktı.

In [3]:
# 0'dan oluşturduğumuz bu vgg16 ağırlıkları sıfırdan atıyoruz yani hiç eğitilmemiş model

# Input tanımlıyoruz
input = Input(shape=(224,224,3))

# Conv1
# filtre adeti, filtre boyutu, stride
x=Conv2D(64,kernel_size=(3,3),padding='same',activation='relu')(input)
x=Conv2D(64,kernel_size=(3,3),padding='same',activation='relu')(x)
x=MaxPooling2D(pool_size=(2,2),strides=(2,2))(x)

x=Conv2D(128,kernel_size=(3,3),padding='same',activation='relu')(x)
x=Conv2D(128,kernel_size=(3,3),padding='same',activation='relu')(x) #vgg'de padding same'dir daralma olacaksa pooling'de oluyor.!
x=MaxPooling2D(pool_size=(2,2),strides=(2,2))(x)

x=Conv2D(256,kernel_size=(3,3),padding='same',activation='relu')(x)
x=Conv2D(256,kernel_size=(3,3),padding='same',activation='relu')(x)
x=Conv2D(256,kernel_size=(3,3),padding='same',activation='relu')(x)
x=MaxPooling2D(pool_size=(2,2),strides=(2,2))(x)

x=Conv2D(512,kernel_size=(3,3),padding='same',activation='relu')(x)
x=Conv2D(512,kernel_size=(3,3),padding='same',activation='relu')(x)
x=Conv2D(512,kernel_size=(3,3),padding='same',activation='relu')(x)
x=MaxPooling2D(pool_size=(2,2),strides=(2,2))(x)

x=Flatten()(x)

x=Dense(4096,activation='relu')(x)
x=Dense(4096,activation='relu')(x)
output=Dense(1000,activation='softmax')(x)

vgg16=Model(inputs=input,outputs=output)
vgg16.summary()

from keras.utils.vis_utils import plot_model
plot_model(vgg16,to_file='model_plot.png',show_shapes=True,show_layer_names=True)

# Bu modeli kodladık, kodlamadan ağırlıklarını nasıl öğrenebiliriz.
# keras.io/api/applications/  dan bak orda yazar.

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 64)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     147584    
                                                             

ModuleNotFoundError: ignored

In [7]:
from keras.utils import load_img, img_to_array
import numpy as np
from keras.applications import VGG16  #ağırlıkları imagenetten alıyoruz yani eğitilmiş model alıyoruz.

img=load_img('./drive/MyDrive/Deep Learning/pakize.jpg',target_size=(224,224))
#veriyi çevirmemiz lazım
img=img_to_array(img)
print(img.shape)

img=img.reshape(1,img.shape[0],img.shape[1],img.shape[2])

#modeli aldık
tf_vgg16=VGG16(weights='imagenet',include_top=True)  #include_top : sen sınıflandırıcı kısmı istiyor musun?

tf_vgg16.summary()

from keras.applications.vgg16 import preprocess_input, decode_predictions

#modele görüntüyü verelim
img=preprocess_input(img)

#modele tahmin ettirelim
pred=tf_vgg16.predict(img)

# en yüksek tahmin sınıfını veriyor.
result=np.argmax(pred,axis=-1)
print(result)

#bu bize en yüksek 5 tahmini veriyor.
label=decode_predictions(pred)
print(label)

# burda eğitilmiş modele verdik,
# kendimiz problemimize göre eğitmek istersek?

(224, 224, 3)
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128

In [ ]:
# Bunun için veri alıp okuyoruz.

train_path='./drive/MyDrive/Deep Learning/chest_xray/train'
test_path='./drive/MyDrive/Deep Learning/chest_xray/test'

trainGen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    vertical_flip=True,
    horizontal_flip=True,
    zoom_range=0.5,
    validation_split=0.1      # validation verisi ayırdı.
)

valGen=ImageDataGenerator(rescale=1./255)
testGen=ImageDataGenerator(rescale=1./255)

# Veri okuma yöntemleri
# flow : görüntüleri okursunuz, listeye atarsanız, liste içerisinden flow(okuyabilirsiniz) atabilirsiniz
# flow_from_directory : bir directory'den oku listeye atma, ihtiyacım olduğunda getir

trainData=trainGen.flow_from_directory(
    train_path,               # bu directory'den oku
    target_size=(227,227),    # hedef ne olsun, tüm görüntüler 28'e 28 olsun dedik, ne kadar büyük okursak o kadar kaliteli görüntü çıkar.
    class_mode= 'binary',     # okuma türünü belirliyoruz  -categorical dersek= 2D one-hut encoding yapılmış şekilde okur, -binary dersek= binary şekilde bir bir okur
    batch_size=8,
    color_mode='rgb',
    subset='training'
    )

valData=trainGen.flow_from_directory(
    test_path,
    target_size=(227,227),
    class_mode= 'binary',
    batch_size=8,
    color_mode='rgb',
    subset='validation'      # ayırdığımız validation datayı kullandık.
    )

testData=trainGen.flow_from_directory(
    test_path,
    target_size=(227,227),
    class_mode= 'binary',
    batch_size=8,
    color_mode='rgb',
    )

Found 4695 images belonging to 2 classes.
Found 62 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
base_model=VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))
#base_model.summary()
# 14milyon parametre veriyor : 124milyon params sınıflandırma, 14 milyon param öğrenmesi için gerekli
# bizim prob. 124 milyon sınıflandırıcı yok ise almamıza gerek yok kendimiz oluştururuz.

# eğer sadece sınıflandırıcı katmanı eğitmek istiyorsak en üstte tanımladıktan sonra(2.satıra) base_model.trainable=False yapacağız
base_model.trainable=False

# peki bir kısmını eğitsek bir kısmını eğitmesek? onu nasıl ayarlayacağız.(örn ilk 3 conv eğiticez, aradaki 3ü eğitmicez gibi vsvs.)
# örneğin ilk 3-8 arasındaki katmanları eğitime katmıyoruz.
#for layer in base_model.layers[4:9]:
#  layer.trainable=False;


# base model output flatten önceki katmanı getirecek ve biz flat edelim.
x=Flatten()(base_model.output)

x=Dense(64,activation='relu')(x)
output=Dense(1,activation='sigmoid')(x)

new_model=Model(inputs=base_model.input,outputs=output)
# ve bakın total param 138 milyondan 16milyona düştü
new_model.summary()

# hangi katmanlar eğitilecek, hangileri eğitilmeyecek : bunları ayarlamamız gerekiyor.
# hangi katmanlar eğitilebilir durumda nasıl görebilirim? :
for layer in new_model.layers:
  print(layer.name," ",layer.trainable)

#Modeli derleyelim
new_model.compile(loss='binary_crossentropy',optimizer='sgd',metrics=['accuracy'])

#Modeli eğitelim
history=new_model.fit(trainData,epochs=15,validation_data=valData)

58889256/58889256 [==============================] - 3s 0us/step
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                            

KeyboardInterrupt: ignored